In [ ]:
from minio.error import S3Error
import fitz  # PyMuPDF
from datetime import datetime
from io import BytesIO
import pandas as pd
import re
import psycopg2
import import_ipynb
from Utils import conectar_minio, guardar_df_en_minio, generar_ruta_fecha
import os
from dotenv import load_dotenv

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']
Conexión exitosa a PostgreSQL PostgreSQL 16.0, compiled by Visual C++ build 1935, 64-bit
2025/05/06


In [ ]:
# Cargar variables del archivo .env
load_dotenv()

In [ ]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']


In [ ]:
def extract_pdf_content(pdf_document):
    """
    Extrae contenido específico de un archivo PDF ya abierto.

    Args:
        pdf_document (fitz.Document): Documento PDF abierto con PyMuPDF.

    Returns:
        dict: Diccionario con la información extraída.
    """
    extracted_data = {
        "test": None,
        "Nombre": None,
        "N. peticion": None,
        "Identificacion": None,
        "Genero": None,
        "Edad": None,
        "Número celular": None,
        "Médico tratante": None,
        "Entidad remitente": None,
        "Entidad": None,
        "Fecha de Ingreso": None,
        "Fecha Informe": None,
        "Tipo estudio": None,
        "Organo": None,
        "Fecha toma": None,
        "Descripcion Macroscopica": None,
        "Descripcion Microscopica": None,
        "Diagnostico": None
    }

    try:
        # Leer cada página del PDF
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text = page.get_text("text")
            cleaned_text = " ".join(text.split())
            
            # Extraer información utilizando búsquedas
            if "-Test de aliento" in cleaned_text:
                extracted_data["test"] = cleaned_text.split("-Test de aliento")[1].strip()#.split("Identificacion")[0].strip()
            
            if "Nombre :" in cleaned_text:
                extracted_data["Nombre"] = cleaned_text.split("Nombre :")[1].split("N. peticion :")[0].strip()
            if "N. peticion :" in cleaned_text:
                extracted_data["N. peticion"] = cleaned_text.split("N. peticion :")[1].split("N.Identificación :")[0].strip()
            if "N.Identificación :" in cleaned_text:
                extracted_data["Identificacion"] = cleaned_text.split("N.Identificación :")[1].split("Genero :")[0].strip()
            if "Genero :" in cleaned_text:
                extracted_data["Genero"] = cleaned_text.split("Genero :")[1].split("Edad :")[0].strip()
            if "Edad :" in cleaned_text:
                extracted_data["Edad"] = cleaned_text.split("Edad :")[1].split("Número celular :")[0].strip()
            if "Número celular :" in cleaned_text:
                extracted_data["Número celular"] = cleaned_text.split("Número celular :")[1].split("Médico tratante :")[0].strip()
            if "Médico tratante :" in cleaned_text:
                extracted_data["Médico tratante"] = cleaned_text.split("Médico tratante :")[1].split("Entidad remitente :")[0].strip()            
            if "Entidad remitente :" in cleaned_text:
                extracted_data["Entidad remitente"] = cleaned_text.split("Entidad remitente :")[1].split("Entidad :")[0].strip()
            if "Entidad :" in cleaned_text:
                extracted_data["Entidad"] = cleaned_text.split("Entidad :")[1].split(" Fecha Ingreso :")[0].strip()
            if "Fecha Ingreso :" in cleaned_text:
                extracted_data["Fecha de Ingreso"] = cleaned_text.split(" Fecha Ingreso :")[1].split("Fecha Informe :")[0].strip()
            if "Fecha Informe :" in cleaned_text:
                extracted_data["Fecha Informe"] = cleaned_text.split("Fecha Informe :")[1].split("Estudios solicitados")[0].strip()
           
            if "Tipo estudio" in cleaned_text:
                extracted_data["Tipo estudio"] = cleaned_text.split("Tipo estudio")[1].split("INFORME DE ANATOMÍA PATOLÓGICA")[0].strip()            
            if "Organo" in cleaned_text:
                extracted_data["Organo"] = cleaned_text.split("Organo")[1].split("INFORME DE ANATOMÍA PATOLÓGICA")[0].strip()
            if "Fecha toma" in cleaned_text:
                extracted_data["Fecha toma"] = cleaned_text.split("Fecha toma")[1].split("INFORME DE ANATOMÍA PATOLÓGICA")[0].strip()
            
            if "DESCRIPCIÓN MACROSCÓPICA" in cleaned_text:
                extracted_data["Descripcion Macroscopica"] = cleaned_text.split("DESCRIPCIÓN MACROSCÓPICA")[1].split("DESCRIPCIÓN MICROSCÓPICA")[0].strip()
            if "DESCRIPCIÓN MICROSCÓPICA" in cleaned_text:
                extracted_data["Descripcion Microscopica"] = cleaned_text.split("DESCRIPCIÓN MICROSCÓPICA")[1].split("DIAGNOSTICO HISTOPATOLOGICO")[0].strip()
            if "DIAGNÓSTICO" in cleaned_text:                
                extracted_data["Diagnostico"] = cleaned_text.split("DIAGNÓSTICO")[1].split("Powered by")[0].strip()

    except Exception as e:
        print(f"Error al extraer datos del PDF: {e}")

    return extracted_data

In [ ]:
import pandas as pd
import fitz  # PyMuPDF

def process_all_pdfs_in_minio(minio_client, bucket_name, folder_path):
    """
    Procesa todos los archivos PDF en una ruta específica de un bucket MinIO.
    
    Parámetros:
    -----------
    minio_client : Minio
        Cliente MinIO ya conectado
    bucket_name : str
        Nombre del bucket donde están los archivos
    folder_path : str
        Ruta dentro del bucket (ej: 'informes/2023/')
    
    Retorna:
    --------
    pandas.DataFrame
        DataFrame con los datos extraídos de todos los PDFs
    """
    results = []
    
    try:
        # Asegurar que la ruta termine con '/'
        if not folder_path.endswith('/'):
            folder_path += '/'
        
        # Listar objetos en la ruta especificada
        objects = minio_client.list_objects(bucket_name, prefix=folder_path, recursive=True)
        
        for obj in objects:
            if obj.object_name.lower().endswith('.pdf'):
                print(f"Procesando archivo: {obj.object_name}")
                pdf_data = None
                pdf_document = None
                
                try:
                    # Obtener el objeto PDF
                    response = minio_client.get_object(bucket_name, obj.object_name)
                    pdf_data = BytesIO(response.read())
                    
                    # Abrir el PDF
                    pdf_document = fitz.open(stream=pdf_data, filetype="pdf")
                    
                    # Extraer contenido (asumiendo que tienes esta función)
                    content = extract_pdf_content(pdf_document)
                    
                    # Agregar los datos al resultado
                    results.append({
                        "test":content.get("test"),
                        "Archivo": obj.object_name.split('/')[-1],  # Solo el nombre del archivo
                        "Ruta Completa": obj.object_name,
                        "Nombre": content.get("Nombre"),
                        "N. Peticion": content.get("N. peticion"),
                        "Identificacion": content.get("Identificacion"),
                        "Genero": content.get("Genero"),
                        "Edad": content.get("Edad"),
                        "Número Celular": content.get("Número celular"),
                        "Médico Tratante": content.get("Médico tratante"),
                        "Entidad Remitente": content.get("Entidad remitente"),
                        "Entidad": content.get("Entidad"),
                        "Fecha de Ingreso": content.get("Fecha de Ingreso"),
                        "Fecha Informe": content.get("Fecha Informe"),
                        "Tipo Estudio": content.get("Tipo estudio"),
                        "Organo": content.get("Organo"),
                        "Fecha Toma": content.get("Fecha toma"),
                        "Descripcion Macroscopica": content.get("Descripcion Macroscopica"),
                        "Descripcion Microscopica": content.get("Descripcion Microscopica"),
                        "Diagnostico": content.get("Diagnostico")
                    })
                    
                except Exception as e:
                    print(f"Error procesando {obj.object_name}: {e}")
                finally:
                    if pdf_document is not None:
                        pdf_document.close()
                    if 'response' in locals():
                        response.close()
                        response.release_conn()
    
    except S3Error as e:
        print(f"Error al acceder al bucket {bucket_name}: {e}")
    except Exception as e:
        print(f"Error inesperado: {e}")
    
    # Convertir resultados en un DataFrame
    return pd.DataFrame(results)

In [ ]:
# Ejemplo de uso
df_resultados = process_all_pdfs_in_minio(
    minio_client=minio_client,  # Tu cliente MinIO ya conectado
    bucket_name="yachay-landing",
    folder_path="Laboratorios/8.PATOLOGOS ASOCIADOS"  # Ruta dentro del bucket
)

# Mostrar resultados
print(f"Se procesaron {len(df_resultados)} archivos PDF")
display(df_resultados.head())

Procesando archivo: Laboratorios/Laboratorio Jimenes Pasto/Q17-23-87245408.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Pasto/Q29-23-5378719.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Pasto/Q33-23-12954743.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Pasto/Q42-23-87471437.pdf
Procesando archivo: Laboratorios/Laboratorio Jimenes Pasto/Q71-23-30708190.pdf
Se procesaron 5 archivos PDF


,Archivo,Ruta Completa,Numero de Informe,Nombre del Paciente,Identificacion,Edad,Telefono,Fecha de Toma de Muestra,Fecha de Ingreso,Fecha de Informe,Entidad,EPS,Servicio,Muestra Remitida,Descripcion Macroscopica,Descripcion Microscopica,Diagnostico,Comentario
0,Q17-23-87245408.pdf,Laboratorios/Laboratorio Jimenes Pasto/Q17-23-...,Q17-23,FLORESMIRO REALPE NARVAEZ,CC 87245408,61,3117207145,5/01/2023,10/01/2023,19/01/2023,UNIDAD CARDIOQUIRURGICA DE NARIÑO FECHA DE ING...,NUEVA EPS,IMAGENOLOGIA,BIOPSIA,se recibe dos muestras: 1. Rotulado “próstata ...,Estudio microscópico realizado. Ver diagnostic...,[1. CILINDROS DE TEJIDO PROSTATICO LADO DERECH...,None
1,Q29-23-5378719.pdf,Laboratorios/Laboratorio Jimenes Pasto/Q29-23-...,Q29-23,PEDRO ALFONSO PORTILLO INSUASTI,CC 5378719,80,SIN,10/01/2023,10/01/2023,9/02/2023,UNIDAD CARDIOQUIRURGICA DE NARIÑO FECHA DE ING...,NUEVA EPS,IMAGENOLOGIA,BIOPSIA,se recibe dos muestras: 1. Rotulado “biopsia p...,A y B) En los cortes histológicos se reconocen...,[1. LOBULO PROSTATICO DERECHO; BIOPSIA TRANSRE...,None
2,Q33-23-12954743.pdf,Laboratorios/Laboratorio Jimenes Pasto/Q33-23-...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,[REVISION E INMUNOHISTOQUIMICA DE UN BLOQUE DE...,None
3,Q42-23-87471437.pdf,Laboratorios/Laboratorio Jimenes Pasto/Q42-23-...,Q42-23,SILVIO ORLANDO IMBAJOA SACANAMBUY,CC 87471437,55,3233645762,3/01/2023,11/01/2023,13/01/2023,UNIDAD CARDIOQUIRURGICA DE NARIÑO FECHA DE ING...,NUEVA EPS,GASTROENTEROLOGIA,ULCERA,se recibe dos muestras: 1. Rotulado “ulcera gá...,A) En los cortes histológicos se observa mucos...,[1. BORDE DE ULCERA GASTRICA; BIOPSIA:  ADENO...,None
4,Q71-23-30708190.pdf,Laboratorios/Laboratorio Jimenes Pasto/Q71-23-...,Q71-23,BLANCA MARLENI MONTILLA DE TORRES,CC 30708190,66,3164040760,13/01/2023,13/01/2023,23/01/2023,UNIDAD CARDIOQUIRURGICA DE NARIÑO FECHA DE ING...,NUEVA EPS,DERMATOLOGIA,BIOPSIA,"Rotulado “piel de mejilla derecha”, en formol ...",Piel con hiperparaqueratosis. Epidermis irregu...,[PIEL MEJILLA DERECHA: BIOPSIA:  LESION LENTI...,None


In [6]:
ruta_fecha = generar_ruta_fecha()

In [ ]:
# Guardar el DataFrame en MinIO
ruta_minio = guardar_df_en_minio(
    minio_client=minio_client,  # Cliente de la conexión anterior
    df=df_resultados,
    bucket_name="yachay-bronze",
    ruta_destino=f"laboratorios/{ruta_fecha}/laboratorio Patologos Asociados/laboratorio_patologos_asociados",  # La extensión se agregará automáticamente
    formato='csv',  # También puede ser 'csv', 'json' o 'excel'
    crear_bucket=True
)

DataFrame guardado exitosamente en: yachay-bronze/laboratorios/2025/05/06/laboratorio Jimenez Pasto/laboratorio_jimenes_pasto.csv
